In [17]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(3)

from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [18]:
# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/Users/chanju/github/project/mask_filtering_project/2주차/이찬주/LeeChanJu/data/train',
        target_size=(28, 28),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/Users/chanju/github/project/mask_filtering_project/2주차/이찬주/LeeChanJu/data/val',
        target_size=(28, 28),    
        batch_size=32,
        class_mode='categorical')



Found 1291 images belonging to 2 classes.
Found 142 images belonging to 2 classes.


In [ ]:
(train_images, train_labels), (test_images, test_labels) = data.load_data()

import matplotlib.pyplot as plt

plt.figure(figsize=(32,28,28,3))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_generator[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

filter out corrupted images

In [20]:
import os

num_skipped = 0
for folder_name in ("with_mask", "without_mask"):
    folder_path = os.path.join("/Users/chanju/github/project/mask_filtering_project/2주차/이찬주/LeeChanJu/data/train", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname) # PetImages/Cat/63.jpg
        try:
            fobj = open(fpath, "rb")
            # fobj.peek(10)은 10바이트를 가져오는데, 여기서는 보통
            # 헤더를 읽기 위해 전체를 가져온다고 생각해도 무방합니다.
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 0 images


In [21]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

In [22]:
# 모델 구성하기
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), padding='same', input_shape=(28,28,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [23]:
model.summary(0)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0

In [24]:
# 모델 엮기
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
hist = model.fit(train_datagen, Y_train, validation_data=(X_validation, Y_validation),
          epochs=30, batch_size=500)

NameError: name 'Y_train' is not defined

In [ ]:
# 모델 학습시키기
# 모델 학습시키기
model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=10,
    validation_data=test_generator,
    validation_steps=10,
)

In [ ]:
# 모델 평가하기
print("-- Evaluate --")

In [ ]:
scores = model.evaluate_generator(
            test_generator, 
            steps = 5)

In [ ]:
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
# 모델 예측하기
print("-- Predict --")

output = model.predict(
            test_generator, 
            steps = 5)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)